In [ ]:
import mcstasscript as ms
import make_powder_instrument
import quizlib

In [ ]:
quiz = quizlib.Powder_Quiz()

In [ ]:
my_configurator = ms.Configurator()
my_configurator.set_mcrun_path("/usr/bin/")
my_configurator.set_mcstas_path("/usr/share/mcstas/3.4/")

# Powder diffraction exercise

In this notebook you will work with a McStas model of a simplified powder diffraction instrument.
You will have to answer questions in the notebook by working with this model, both by running simulations and expanding the model.
We will use the Python McStas API McStasScript to work with the instrument, you can find documentation [here](https://mads-bertelsen.github.io).

## Get the instrument object
First we need the McStas instrument object. Here it is retrieved from a local python function that generates it.

In [ ]:
instrument = make_powder_instrument.make(input_path="run_folder")

## Investigate instrument
The first task is to investigate the instrument object `instrument` using some of the available methods available on that object. Each method that show something about the instrument starts with the word show, so you can use tab to autocomplete in the cell to see the relevant methods.

In particular, look at what parameters are available and take a look at the instrument geometry.

In [ ]:
instrument.show_parameters()

In [ ]:
instrument.show_instrument()

## Set parameters

Before we run a simulation using the instrument, we need to set some parameters to the desired values.

## Instrument settings
Before running the simulation, a few settings pertaining to computing options need to be specified. This is done with a different method to clearly distinguish these from the instrument parameters. One important setting is called `output_path` which sets the name of the generated folder with simulation output.

In [ ]:
instrument.settings(ncount=5e6, mpi=4, suppress_output=True, NeXus=True, output_path="first_run")

In [ ]:
instrument.settings(mpi=2)

## Run the instrument
Now the simulation can be executed with the `backengine` method. This method returns a list of data object. Store this returned data in a python variable named `data`.

In [ ]:
data = instrument.backengine()
data

### Visualize the data
The data objects in the returned list can be plotted with the McStasScript function `make_plot`. The plots can be customized, use these keyword arguments:
- log : True
- orders_of_mag : 5 (maximum orders of magnitudes plotted when using logarithmic plotting)

In [ ]:
ms.make_sub_plot(data, log=True, orders_of_mag=5)

## Improve the instrument


## Run improved instrument

## Run with sample


## Increase the number of pulses

Your final task is to re-run the simulations with and without sample using 3 pulses from the source instead of 1 which was used so far.
This is controlled using the *n_pulses* parameter on the instrument object.

**Hints:**

- Change the destination folder so that you don't overwrite the results from the 1-pulse simulations.
- Remember to adjust `ncount` accordingly, we would like 3 times more rays now that we use 3 pulses.

In [ ]:
# Without sample
instrument.settings(ncount=1.5e7, output_path="powder_without_sample_3_pulse")
instrument.set_parameters(enable_sample=0, n_pulses=3, integration_time=5e4)
background_3_pulses = instrument.backengine()

# With sample
instrument.settings(ncount=1.5e7, output_path="powder_with_sample_3_pulse")
instrument.set_parameters(enable_sample=1, n_pulses=3, integration_time=500)
sample_3_pulses = instrument.backengine()

### Compare results with shorter runs
By saving the results in different python variables, you can compare the two runs by plotting each. Use of the `make_sub_plot` function might make it easier to view each monitor side by side.

In [ ]:
def compare_monitors(monitor_name):
    sample_low_ncount = ms.name_search(monitor_name, sample_data)
    sample_high_ncount = ms.name_search(monitor_name, sample_3_pulses)
    ms.make_sub_plot([sample_low_ncount, sample_high_ncount], log=True, orders_of_mag=5)    

compare_monitors("signal")

In [ ]:
compare_monitors("signal_tof_all")

In [ ]:
compare_monitors("signal_tof")